In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
%cd /content/drive/MyDrive/proyectos3/
!ls

/content/drive/MyDrive/proyectos3
alexNet.ipynb  PlantDoc  PlantVillage_bgremoved  resNet.ipynb  wandb


# Cargar Conjunto de Datos

In [37]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import ConcatDataset


train_dir0 = './PlantVillage_bgremoved/train'
train_dir1 = './PlantDoc/train/'

val_dir0 = './PlantVillage_bgremoved/valid'
val_dir1 = './PlantDoc/test/'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset0 = ImageFolder(train_dir0, transform=transform,  is_valid_file=lambda x: x.endswith('.JPG'))
train_dataset1 = ImageFolder(train_dir1, transform=transform)

val_dataset0 = ImageFolder(val_dir0, transform=transform, is_valid_file=lambda x: x.endswith('.JPG'))
val_dataset1 = ImageFolder(val_dir1, transform=transform)

train_dataset = ConcatDataset([train_dataset0, train_dataset1])
val_dataset = ConcatDataset([val_dataset0, val_dataset1])

classes = train_dataset0.classes

classes

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

crear carga de datos para facilitar el acceso a los datos

In [38]:
from torch.utils.data import DataLoader

batch_size = 32
num_workers = 8 # Número de procesos que se utilizarán para cargar datos en paralelo
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [39]:
import torch

def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [40]:
device = get_default_device()

train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

In [41]:
device

device(type='cuda')

In [42]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import precision_score, f1_score

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class ImageClassificationBase(nn.Module):

    def training_step(self, batch):
        "calculate loss for a batch of training data"
        images, labels = batch
        out = self(images)                  # Generate predictions
        # nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        "calculate loss, accuracy, precision and f1 score for a batch of validation data"
        images, labels = batch
        out = self(images)                   # Generate prediction
        # nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss
        loss = F.cross_entropy(out, labels)  # Calculate loss
        preds = torch.argmax(out, dim=1)
        acc = accuracy(out, labels)          # Calculate accuracy

        # calculate precision and f1 score
        precision = precision_score(labels.cpu(), preds.cpu(), average='weighted', zero_division=1)
        f1 = f1_score(labels.cpu(), preds.cpu(), average='macro')
        return {'val_loss': loss.detach(), 'val_accuracy': acc, 'val_precision': precision, 'val_f1': f1}

    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()       # Combine loss
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {"val_loss": epoch_loss.item(), "val_accuracy": epoch_accuracy.item()} # Combine accuracies

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_accuracy']))


In [43]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_planes, out_channels=planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=planes)
        self.conv2 = nn.Conv2d(in_channels=planes, out_channels=planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=planes)

        self.relu = nn.ReLU(inplace=True)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels=in_planes, out_channels=self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(num_features=self.expansion * planes)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_planes, out_channels=planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=planes)
        self.conv2 = nn.Conv2d(in_channels=planes, out_channels=planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=planes)
        self.conv3 = nn.Conv2d(in_channels=planes, out_channels=self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(num_features=self.expansion * planes)

        self.relu = nn.ReLU(inplace=True)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels=in_planes, out_channels=self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(num_features=self.expansion * planes)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet(ImageClassificationBase):
    def __init__(self, block, num_blocks, num_classes=3):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.layer1 = self._make_layer(block=block, planes=64, num_blocks=num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block=block, planes=128, num_blocks=num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block=block, planes=256, num_blocks=num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block=block, planes=512, num_blocks=num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(in_features=512 * block.expansion, out_features=num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        layers = []
        layers.append(block(in_planes=self.in_planes, planes=planes, stride=stride))
        self.in_planes = planes * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(in_planes=self.in_planes, planes=planes, stride=1))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.bn1(self.conv1(x))
        out = F.max_pool2d(input=out, kernel_size=3, stride=2, padding=1)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, start_dim=1)
        out = self.fc(out)
        return out

In [44]:
model = to_device(ResNet(block=BasicBlock, num_blocks=[2, 2, 2, 2]), device)

In [45]:
from torchsummary import summary              # for getting the summary of our model

# getting summary of the model
INPUT_SHAPE = (3, 224, 224)
print(summary(model.cuda(), (INPUT_SHAPE)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
            Conv2d-3           [-1, 64, 56, 56]          36,864
       BatchNorm2d-4           [-1, 64, 56, 56]             128
              ReLU-5           [-1, 64, 56, 56]               0
            Conv2d-6           [-1, 64, 56, 56]          36,864
       BatchNorm2d-7           [-1, 64, 56, 56]             128
              ReLU-8           [-1, 64, 56, 56]               0
        BasicBlock-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,864
      BatchNorm2d-11           [-1, 64, 56, 56]             128
             ReLU-12           [-1, 64, 56, 56]               0
           Conv2d-13           [-1, 64, 56, 56]          36,864
      BatchNorm2d-14           [-1, 64,

In [46]:
epochs = 100

max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [47]:
!pip install wandb

In [48]:
import wandb

name = f'resnet-{epochs}-epochs'
wandb.init(project="plantvillage-aug", name=name, config={
    "epochs": epochs,
    "learning_rate": max_lr,
    "grad_clip": grad_clip,
    "weight_decay": weight_decay,
    "opt_func": opt_func.__name__,
    "batch_size": batch_size,
    "dataset": "PlantVillage",
    "architecture": "resnet"
})

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Train Loss,▆▅▅▆▇▃▄▂▅▄▅▄█▆▂▂▅▅▃▂▁▃▂▂▃▃▂▂▁▁▂▁▁▁▁▁▁▁▁▁
Val Accuracy,▂▃▇▅▅▁▅▂▄▂▄█▇▅▅▇▆▆▇▆▆▆▅▆▆▇▆▇▆▆▆▇▇▇▇▇▇▇▇▇
Val F1,▂▂▄▄▅▁▃▁▄▁▄▆▄▅▆▆▆▆▆▇▆▄▅▅▆▆▆▅▆▆▅▇▇▇▇█████
Val Loss,▇█▃▄▄▇▂▄▄▆▆▂▂▂▂▁▅▂▁▂▂▂▄▂▃▁▃▃▄▃▂▁▂▂▁▂▂▃▃▂
Val Precision,▂▂▂▁▂▁▇▃█▃▆▅▄▆█▄▄█▅▅█▅▄▆▆▇▅▆▆▆▆▅▆▆▆▇▇▇▇▇
Epoch,49
Train Loss,0.00412
Val Accuracy,0.91118
Val F1,0.86296
Val Loss,0.3433


In [49]:
# for training
@torch.no_grad()
def evaluate(model, val_loader):
    """Evaluates the model's performance on the validation set"""
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    # return model.validation_epoch_end(outputs)
    val_losses = [x["val_loss"] for x in outputs]
    epoch_loss = torch.stack(val_losses).mean()       # Combine loss
    val_accuracies = [x["val_accuracy"] for x in outputs]
    epoch_accuracy = torch.stack(val_accuracies).mean()
    val_precisions = [x["val_precision"] for x in outputs]
    epoch_precision = torch.tensor(sum(val_precisions)/len(val_precisions))
    val_f1s = [x["val_f1"] for x in outputs]
    epoch_f1 = torch.tensor(sum(val_f1s)/len(val_f1s))
    return {"val_loss": epoch_loss.item(), "val_accuracy": epoch_accuracy.item(), "val_precision": epoch_precision.item(), "val_f1": epoch_f1.item()} # Combine accuracies


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0,
                grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # scheduler for one cycle learniing rate
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))


    for epoch in range(epochs):
        # Training
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # recording and updating learning rates
            lrs.append(get_lr(optimizer))
            sched.step()

            # logging to wandb
            wandb.log({"Train Loss": loss.item()})


        # validation
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)

        # logging to wandb
        wandb.log({
            "Epoch": epoch,
            "Val Loss": result['val_loss'],
            "Val Accuracy": result['val_accuracy'],
            "Val Precision": result['val_precision'],
            "Val F1": result['val_f1']
        })

    return history

In [50]:
%%time
history = fit_one_cycle(
  epochs,
  max_lr,
  model,
  train_loader,
  val_loader,
  grad_clip=grad_clip,
  weight_decay=1e-4,
  opt_func=opt_func
)

Epoch [0], last_lr: 0.00043, train_loss: 0.5772, val_loss: 0.5809, val_acc: 0.7368
Epoch [1], last_lr: 0.00050, train_loss: 0.4012, val_loss: 0.4320, val_acc: 0.8125
Epoch [2], last_lr: 0.00063, train_loss: 0.3504, val_loss: 0.8870, val_acc: 0.7007
Epoch [3], last_lr: 0.00081, train_loss: 0.3704, val_loss: 0.4741, val_acc: 0.8109
Epoch [4], last_lr: 0.00104, train_loss: 0.3763, val_loss: 0.7476, val_acc: 0.7023
Epoch [5], last_lr: 0.00131, train_loss: 0.3351, val_loss: 1.0920, val_acc: 0.6201
Epoch [6], last_lr: 0.00163, train_loss: 0.3753, val_loss: 0.4813, val_acc: 0.7993
Epoch [7], last_lr: 0.00198, train_loss: 0.3030, val_loss: 0.4052, val_acc: 0.8322
Epoch [8], last_lr: 0.00237, train_loss: 0.3612, val_loss: 0.9711, val_acc: 0.6875
Epoch [9], last_lr: 0.00280, train_loss: 0.3642, val_loss: 0.9526, val_acc: 0.6990
Epoch [10], last_lr: 0.00324, train_loss: 0.3363, val_loss: 0.5834, val_acc: 0.7500
Epoch [11], last_lr: 0.00371, train_loss: 0.3198, val_loss: 0.3342, val_acc: 0.8372
Ep

In [51]:
# clear nvidia cache
import torch, gc
gc.collect()
torch.cuda.empty_cache()